In [2]:
#################### PART 2 - Transfer Learning #########################

import pandasql as ps
import tensorflow as tf
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [4]:
#Import dataset, shape it, extract input and target from it
X_Y_new = pd.read_csv('owndataset.csv')
X_Y_new['reviews']=X_Y_new['reviews'].str.replace('<br />', '').replace('[0-9]', '')
train, validate= np.split(X_Y_new.sample(frac=1), [int(.7*len(X_Y_new))])
x_train = (train.reviews) 
y_train= (train.sentiment)
x_validate = (validate.reviews)
y_validate = (validate.sentiment)


In [6]:
#Tokenise the input sequences
from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=1000,filters='![0-9]"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=False)
tokenizer.fit_on_texts(X_Y_new['reviews'])

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Lenovo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-b260c576d97f>", line 2, in <module>
    from keras.preprocessing.text import text_to_word_sequence
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python37\site-packages\keras\__init__.py", line 3, in <module>
    from . import utils
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python37\site-packages\keras\utils\__init__.py", line 6, in <module>
    from . import conv_utils
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python37\site-packages\keras\utils\conv_utils.py", line 9, in <module>
    from .. import backend as K
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python37\site-packages\keras\backend\__init__.py", line 1, in <module>
    from .load_backend import epsilon
  File "C:\Users\Lenovo\AppData\Roaming\Python\Python37\site-packages\keras\backend\load_bac

FileNotFoundError: [Errno 2] File b'/home/pmohan/olympus/Customer_Match_Code_OLYMPUS_0003_part_00' does not exist: b'/home/pmohan/olympus/Customer_Match_Code_OLYMPUS_0003_part_00'

In [66]:
#Decide the vocab length
unique_vocab = len(tokenizer.word_index)
unique_vocab

511

In [67]:
#MAximum length of each sequence
maxLen = len(max(X_Y_new.reviews, key=len).split(' '))
print(maxLen)

44


In [68]:
#Tokenise Training sequences
x_train = tokenizer.texts_to_sequences(x_train)
print(x_train[0])
x_train = sequence.pad_sequences(x_train, maxlen=maxLen)

[11, 263, 40, 264, 265, 2, 266, 4, 267, 1, 2, 268, 269, 73, 2, 96, 4, 2, 6, 5, 94, 270]


In [69]:
#Tokenise validation sequences
x_validate = tokenizer.texts_to_sequences(x_validate)
x_validate = sequence.pad_sequences(x_validate, maxlen=maxLen)

In [70]:
#Prepare target labels
y_train_new = ps.sqldf("select case when sentiment='positive' then 1 else 0 end as sentiment_new from train")
y_validation_new = ps.sqldf("select case when sentiment='positive' then 1 else 0 end as sentiment_new from validate")

In [71]:
#convert into numpy arrays for fast processing
y_train_new = np.asarray(y_train_new['sentiment_new'])
y_validation_new = np.asarray(y_validation_new['sentiment_new'])

In [72]:
#needed modules
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.datasets import imdb
import tensorflow as tf

In [79]:
#model design to math the previous best architecture (2 LSTM)
model = Sequential()
embedding_size=50
model.add(Embedding(unique_vocab,
                    embedding_size,
                    input_length=maxLen))

model.add(tf.keras.layers.LSTM(100,return_sequences=True,  stateful=False,recurrent_initializer='glorot_uniform'))
model.add(tf.keras.layers.LSTM(100,return_sequences=False, stateful=False,recurrent_initializer='glorot_uniform'))
model.add(Dense(100,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 44, 50)            25550     
_________________________________________________________________
lstm_27 (LSTM)               (None, 44, 100)           60400     
_________________________________________________________________
lstm_28 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_48 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 101       
Total params: 176,551
Trainable params: 176,551
Non-trainable params: 0
_______________________________________________

In [80]:
#Loading weights from previous best 2 lstm moidel
#the below line alone will be commented in case of building a scratch model
model.load_weights(tf.train.latest_checkpoint('bestt_2_1_lstm2'))
#Extra fine tuning
model.add(Dense(200,activation='relu'))
model.add(Dense(200,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()


Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 44, 50)            25550     
_________________________________________________________________
lstm_27 (LSTM)               (None, 44, 100)           60400     
_________________________________________________________________
lstm_28 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_48 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 101       
_________________________________________________________________
dense_50 (Dense)             (None, 200)             

In [81]:
#Loss Function
from tensorflow.keras import optimizers
AD = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=AD,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [84]:
#Model Evaluation
loss, acc = model.evaluate(x_validate, y_validation_new,batch_size=1)
loss, acc = model.evaluate(x_train, y_train_new,batch_size=1)

41/41 [==============================] - 1s 30ms/sample - loss: 0.0989 - acc: 0.9756


In [82]:
#Apply Early stopping # experimented with and without this
from tensorflow.keras.callbacks import EarlyStopping 
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7) 

In [ ]:
#model training and saving it
hist = model.fit(x_train, y_train_new,
         
          epochs=100,verbose=1,validation_data=(x_validate, y_validation_new)
          )


In [85]:
#model saving
model.save('transfer_Learning.h5')  